## Run Interative Plot to File Widget

In [14]:
import numpy as np
from math import pi
from test_functions1D import MultiMinimaFunc_numpy, Sinc_numpy, Sin_numpy
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, Output, VBox, HTML, HBox, Dropdown
from scipy.optimize import differential_evolution

# Initialize global variables for function and plotting range
selected_function = MultiMinimaFunc_numpy  # Set default function to avoid NoneType error
f_global_min_x = None
f_global_min_value = None
intervalx_a, intervalx_b = -30.0, 30.0
intervaly_a, intervaly_b = None, None

# Create x_k slider with initial range
x_k_slider = FloatSlider(value=(intervalx_a+intervalx_b)/2, min=intervalx_a, max=intervalx_b, step=0.01, description='x_k:')

# Update the function and range based on dropdown selection
def update_function(change):
    global selected_function, f_global_min_x, f_global_min_value, intervalx_a, intervalx_b, intervaly_a, intervaly_b
    
    if change['new'] == 'Sinc':
        selected_function = Sinc_numpy
        intervalx_a, intervalx_b = -20, 20

        # Define global minima for Sinc function
        f_global_min_x = [4.49341, -4.49341]
        f_global_min_value = [selected_function(x) for x in f_global_min_x]
    
    elif change['new'] == 'Sin':
        selected_function = Sin_numpy
        intervalx_a, intervalx_b = -3.5 * pi, 2.5 * pi

        # Calculate global minima positions within the specified range
        f_global_min_x = [3 * pi / 2 + 2 * n * pi for n in range(-4, 3)]
        f_global_min_x = [x for x in f_global_min_x if intervalx_a <= x <= intervalx_b]
        f_global_min_value = [selected_function(x) for x in f_global_min_x]
    
    else:  # 'MultiMinima'
        selected_function = MultiMinimaFunc_numpy
        f_global_min_x = -1.51035
        f_global_min_value = selected_function(f_global_min_x)
        intervalx_a, intervalx_b = -30, 30
    
    # Update x_k slider range
    x_k_slider.min = intervalx_a
    x_k_slider.max = intervalx_b
    x_k_slider.value = (intervalx_a + intervalx_b) / 2  # Reset slider to the middle
    
    # Update plot when function changes
    update_plot(None)

# Define the function h(x)
def h(x, fixed_T, x_k):
    return selected_function(x) + (1 / (2 * fixed_T)) * (x - x_k)**2

# Define the function u(x)
def u(x, x_hat, fixed_T):
    return selected_function(x_hat) + (1 / (2 * fixed_T)) * (x_hat - x)**2

# Function to update the plot based on slider values
def update_plot(button):
    fixed_T = fixed_T_slider.value
    x_k = x_k_slider.value
    alpha = alpha_slider.value

    x_values = np.linspace(intervalx_a, intervalx_b, 400)

    # Find the global minimum using the differential_evolution method
    result = differential_evolution(h, [(intervalx_a, intervalx_b)], args=(fixed_T, x_k))

    f_values = selected_function(x_values)
    h_values = h(x_values, fixed_T, x_k)

    if result.success:
        x_hat = result.x[0]  # Get the value of x_hat from the optimization result
        x_k_plus_1 = x_k - alpha * (x_k - x_hat)  # Compute x_{k+1}
        f_x_k_plus_1 = selected_function(x_k_plus_1)  # Calculate f(x_{k+1})
        u_values = u(x_values, x_hat, fixed_T)

        # Plotting f(x)
        plt.figure(figsize=(12, 8))
        plt.plot(x_values, f_values, label=r'$f(x)$', color='blue')
        plt.plot(x_values, h_values, label=r'$f(x) + \frac{1}{2T} (x - x_k)^2$', color='orange')

        # Plot u(x)
        plt.plot(x_values, u_values, label=r'Moreau Envelope, $u(x,T)$', color='green')

        # Mark the global minimum of h(x)
        global_min_value = result.fun
        plt.scatter(x_hat, global_min_value, color='red', zorder=5, label=r'Proximal, $\hat{x}_k=prox_{Tf}(x_k)\approx x_k-T g_k$')

        # Calculate the minimum of u(x)
        u_min_value = u(x_hat, x_hat, fixed_T)
        plt.scatter(x_hat, u_min_value, color='purple', zorder=5, label='Minimum of $u(x,T)$', marker='x')

    # Calculate the value of f at x_k
    f_xk_value = selected_function(x_k)
    plt.scatter(x_k, f_xk_value, color='orange', zorder=5, label=r'Current Iteration $f(x_k)$', marker='o')

    # Mark the global minimum of f(x)
    plt.scatter(f_global_min_x, f_global_min_value, color='cyan', zorder=5, label=r'Global Minima, $f(x_{true})$', marker='s')

    # Plot the point for f(x_{k+1})
    plt.scatter(x_k_plus_1, f_x_k_plus_1, color='magenta', zorder=5, 
            label=r'Next Iteration $f(x_{k+1})$, for $x_{k+1}=$' + f'{x_k_plus_1:.2f}\nwhere '+r'$x_{k+1}=x_k+\alpha(x_k-\hat{x}_k)$', marker='^')

    # Add titles and labels
    plt.title('Plot To Visualize Moreau Adaptive Descent')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axhline(0, color='black', linewidth=0.5, ls='--')
    plt.axvline(0, color='black', linewidth=0.5, ls='--')

    # Set limits and grid
    plt.xlim(intervalx_a, intervalx_b)

    # Dynamically set the y-limits based on function outputs
    y_min = np.min(f_values)
    y_max = np.max(f_values)

    if y_max < 0:
        # If all values are negative, set the limits to give some visual space
        plt.ylim(1.2 * y_min, 0)  # Set lower limit 20% below min, upper limit at 0
    elif y_min > 0:
        plt.ylim(0.8 * y_min, 1.2 * y_max)  # 20% less than the min if min is positive
    else:
        plt.ylim(1.2 * y_min, 1.2 * y_max)  # 20% more than the min if min is zero or negative

    plt.grid(which='major', linestyle='-', linewidth='0.5')
    plt.minorticks_on()
    plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(1))
    plt.grid(which='minor', linestyle=':', linewidth='0.5')
    plt.legend(loc='upper right')

    # Save the figure as a PNG file
    plt.savefig("interactive_plot.png", format='png', bbox_inches='tight')
    plt.close()  # Close the plot to free up memory

# Function to update x_k to x_k_plus_1 and replot
def next_step(button):
    fixed_T = fixed_T_slider.value
    x_k = x_k_slider.value
    alpha = alpha_slider.value

    # Find the global minimum using the differential_evolution method
    result = differential_evolution(h, [(intervalx_a, intervalx_b)], args=(fixed_T, x_k))

    if result.success:
        x_hat = result.x[0]
        x_k_plus_1 = x_k - alpha * (x_k - x_hat)
        
        # Update the x_k slider value to the new x_k_plus_1
        x_k_slider.value = x_k_plus_1

        # Replot using the updated x_k
        update_plot(button)

# Create sliders
fixed_T_slider = FloatSlider(value=10.0, min=0.5, max=100.0, step=0.5, description='Fixed time, T:')
alpha_slider = FloatSlider(value=1.0, min=0.1, max=1.9, step=0.1, description=r'Step Size:')

# Create function dropdown
function_dropdown = Dropdown(
    options=['Sinc', 'Sin', 'MultiMinima'],
    value='MultiMinima',
    description='Function:'
)

# Bind dropdown change event
function_dropdown.observe(update_function, names='value')

# Create buttons
update_button = Button(description="Update Plot")
next_step_button = Button(description="Next Step")

# Connect the buttons to their respective functions
update_button.on_click(update_plot)
next_step_button.on_click(next_step)

# Create a note explaining the sliders and the plot
note = HTML("""<h3>Instructions:</h3>
<p style="margin-bottom: 5px;">Adjust the sliders to modify the values of fixed time <strong>T</strong>, <strong>x_k</strong>, and Step Size <strong>&alpha;</strong>.</p>
<p style="margin-bottom: 5px;">Select a function from the dropdown. The available functions are:</p>
<ul>
    <li><strong>Sinc:</strong> This function has two global minimum and oscillates.</li>
    <li><strong>Sin:</strong> This periodic function has periodic global minima.</li>
    <li><strong>MultiMinima:</strong> This function features several local minima and a global.</li>
</ul>
<p style="margin-bottom: 5px;">After selecting a function, press the update button to visualize the effects of your adjustments.</p>
<p style="margin-bottom: 5px;">Running the plot updates the file <strong>interactive_plot.png</strong> with the current visualization.</p>
""")

# Create a title for the display
title = HTML("<h2>Moreau Adaptive Descent Visualization (for Fixed Time T in 1D)</h2>")

# Display title, sliders, dropdown, and buttons
display(VBox([title, function_dropdown, fixed_T_slider, x_k_slider, alpha_slider, HBox([update_button, next_step_button]), note]))
